In [150]:
import numpy as np
from utils import get_rand_scale_vec
from lookup import features, frame_names, contraint_types

In [151]:
# Main motions
max_phases = 4
max_phase_contraints = 4
target_prob = .7
scale_as_vec_prob = .2
slices = 32
accumulated_collisions = False

# Sub-motions
uniform_mode_switch_prob = .1
max_mode_switches = 2

In [152]:
phases_count = np.random.randint(1, max_phases+1)
mode_switch_count = np.random.randint(0, max_mode_switches+1)

final_komo = ""
final_komo += f"komo = ry.KOMO(C, {phases_count}, {slices}, 2, {accumulated_collisions})\n"
final_komo += f"komo.addControlObjective([], 0, 1e-2)\n"
final_komo += f"komo.addControlObjective([], 1, 1e-1)\n"
final_komo += f"komo.addControlObjective([], 2, 1e0)\n"
final_komo += f"komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)\n"
final_komo += f"komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)\n"

In [153]:
# Main phases
for i in range(phases_count):
    
    repeats = np.random.randint(1, max_phase_contraints+1) # TODO: Make this non-uniform

    for _ in range(repeats):
        feature_name = np.random.choice(list(features.keys()))
        contraint_type = np.random.choice(contraint_types)
        scale_pow = np.random.randint(-2, 3) # As matrix?

        scale_as_vec = np.random.random() < scale_as_vec_prob
        if scale_as_vec:
            scale_vec = get_rand_scale_vec(features[feature_name][1])
            scale = "["
            for v in scale_vec:
                if v == 1:
                    scale += f"1e{scale_pow}, "
                else:
                    scale += "0, "
            scale += "]"
            scale = scale.replace(", ]", "]")
        else:
            scale = f"[1e{scale_pow}]"
        
        frames_count = features[feature_name][0]
        frames = np.random.choice(frame_names, frames_count, replace=False).tolist()

        specific_target = np.random.random() < target_prob
        if specific_target:
            target_range = np.pi*2
            target = np.round(np.random.uniform(-target_range, target_range, (features[feature_name][1])), 3).tolist()
            final_komo += f"komo.addObjective([{i+1}], ry.FS.{feature_name}, {frames}, ry.OT.{contraint_type}, {scale}, {target})\n"
        else:
            final_komo += f"komo.addObjective([{i+1}], ry.FS.{feature_name}, {frames}, ry.OT.{contraint_type}, {scale})\n"

# Mode Switches
if phases_count >= 2:
    for i in range(mode_switch_count):
        first_switch = i == 0
        if np.random.random() < uniform_mode_switch_prob:
            start = np.random.uniform(0, phases_count-.1)
            start = np.round(start, 1)
            end = np.random.uniform(start+.1, phases_count)
            end = np.round(end, 1)
        else:
            start = np.random.randint(0, phases_count)
            end = np.random.randint(start+1, phases_count+1)
            
        frames = np.random.choice(frame_names, 2, replace=False).tolist()
        final_komo += f"komo.addModeSwitch([{start}, {end}], ry.SY.stable, {frames}, {first_switch})\n"

In [154]:
print(final_komo)

komo = ry.KOMO(C, 2, 32, 2, False)
komo.addControlObjective([], 0, 1e-2)
komo.addControlObjective([], 1, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([1], ry.FS.scalarProductYZ, ['box2', 'l_gripper'], ry.OT.sos, [1e-2])
komo.addObjective([2], ry.FS.poseDiff, ['box1', 'box2'], ry.OT.eq, [0, 0, 1e2, 1e2, 0, 0, 1e2], [1.444, -3.594, 4.474, 5.851, 2.849, 1.348, 4.947])
komo.addObjective([2], ry.FS.scalarProductXZ, ['box2', 'handle'], ry.OT.eq, [1e-2], [0.296])
komo.addObjective([2], ry.FS.scalarProductYY, ['l_gripper', 'box2'], ry.OT.eq, [1e-2], [1.588])
komo.addModeSwitch([1, 2], ry.SY.stable, ['l_gripper', 'box2'], True)

